# 제품 이상여부 판별 프로젝트


## 데이터 불러오기


### 필수 라이브러리


In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
from pprint import pprint

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

### 데이터 읽어오기


In [40]:
import pandas as pd

THRESHOLD = 0.3
RANDOM_STATE = 110

train_data = pd.read_csv("train_data_0827.csv")
test_data = pd.read_csv("test_data_0827.csv")

In [41]:
# dam, fill1, fill2 공통 변수
var_dam_fill = [
    'Receip_No_encoded',
    'Equipment_same_num',
    'PalletID_Collect_Result_encoded',
    'Production_Qty_Collect_Result',
    'WorkMode Collect Result'
]

In [42]:
# 전체 공통 변수
### train
var_all_train = [
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

### test
var_all_test = [
    'Set ID',
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

In [43]:
# '_Dam'을 포함하는 변수 선택
dam_variables = [var for var in train_data.columns if '_Dam' in var]

# train
final_columns_train = var_dam_fill + var_all_train + dam_variables
train_data_dam = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + dam_variables
test_data_dam = test_data[final_columns_test]

In [44]:
# '_Fill1'을 포함하는 변수 선택
fill1_variables = [var for var in train_data.columns if '_Fill1' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill1_variables
train_data_fill1 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill1_variables
test_data_fill1 = test_data[final_columns_test]

In [45]:
# '_Fill2'을 포함하는 변수 선택
fill2_variables = [var for var in train_data.columns if '_Fill2' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill2_variables
train_data_fill2 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill2_variables
test_data_fill2 = test_data[final_columns_test]

In [46]:
# '_AutoClave'을 포함하는 변수 선택
autoclave_variables = [var for var in train_data.columns if '_AutoClave' in var]

# train
final_columns_train = var_all_train + autoclave_variables
train_data_autoclave = train_data[final_columns_train]

# test 
final_columns_test = var_all_test + autoclave_variables
test_data_autoclave = test_data[final_columns_test]

In [47]:
# 각 DataFrame의 칼럼 수 계산
num_columns_train_data = train_data.shape[1]
num_columns_train_data_dam = train_data_dam.shape[1]
num_columns_train_data_autoclave = train_data_autoclave.shape[1]
num_columns_train_data_fill1 = train_data_fill1.shape[1]
num_columns_train_data_fill2 = train_data_fill2.shape[1]

num_columns_test_data = test_data.shape[1]
num_columns_test_data_dam = test_data_dam.shape[1]
num_columns_test_data_autoclave = test_data_autoclave.shape[1]
num_columns_test_data_fill1 = test_data_fill1.shape[1]
num_columns_test_data_fill2 = test_data_fill2.shape[1]

# 각 DataFrame의 칼럼 수 출력
print("----train data-----")
print(f"train_data DataFrame의 칼럼 수: {num_columns_train_data}")
print(f"train_data_dam DataFrame의 칼럼 수: {num_columns_train_data_dam}")
print(f"train_data_autoclave DataFrame의 칼럼 수: {num_columns_train_data_autoclave}")
print(f"train_data_fill1 DataFrame의 칼럼 수: {num_columns_train_data_fill1}")
print(f"train_data_fill2 DataFrame의 칼럼 수: {num_columns_train_data_fill2}")
print("----test data-----")
print(f"test_data DataFrame의 칼럼 수: {num_columns_test_data}")
print(f"test_data_dam DataFrame의 칼럼 수: {num_columns_test_data_dam}")
print(f"test_data_autoclave DataFrame의 칼럼 수: {num_columns_test_data_autoclave}")
print(f"test_data_fill1 DataFrame의 칼럼 수: {num_columns_test_data_fill1}")
print(f"test_data_fill2 DataFrame의 칼럼 수: {num_columns_test_data_fill2}")

----train data-----
train_data DataFrame의 칼럼 수: 40
train_data_dam DataFrame의 칼럼 수: 23
train_data_autoclave DataFrame의 칼럼 수: 8
train_data_fill1 DataFrame의 칼럼 수: 14
train_data_fill2 DataFrame의 칼럼 수: 14
----test data-----
test_data DataFrame의 칼럼 수: 41
test_data_dam DataFrame의 칼럼 수: 24
test_data_autoclave DataFrame의 칼럼 수: 9
test_data_fill1 DataFrame의 칼럼 수: 15
test_data_fill2 DataFrame의 칼럼 수: 15


---

## 모델링

### 모델 정의

In [48]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.ensemble import VotingClassifier

# 스레드홀드 설정
THRESHOLD = 0.3

# 모델 설정 및 하이퍼파라미터
models = {
    'et': ExtraTreesClassifier(),
    'rf': RandomForestClassifier(),
    'cat': CatBoostClassifier(),
    'lgbm': LGBMClassifier(),
    'xgb': XGBClassifier(),
    'dt': DecisionTreeClassifier(),
    'ada': AdaBoostClassifier()
}


def train_and_evaluate_model(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return
    
    # 데이터셋 분할
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # 모델 선택
    model = models[model_name].__class__()  # 새로운 모델 인스턴스 생성

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(x_train, y_train)

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    # 예측
    y_val_pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    y_val_pred = (y_val_pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측
    
#     fn_indices = np.where((y_val == 1) & (y_val_pred == 0))[0]
#     fn_array = np.array(fn_indices)


    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, zero_division=0)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'{model_name} 모델이 {data_name} 데이터로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

    return model # 학습된 모델 반환

def fit_all_train_data_function(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return None  # 지원되지 않는 모델일 경우 None 반환
    
    # 모델 선택
    model = models[model_name].__class__()  # 새로운 모델 인스턴스 생성

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(data.drop("target", axis=1), data["target"].map({'Normal': 0, 'AbNormal': 1}))

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    print(f'{model_name} 모델이 {data_name} 데이터로 학습 완료')
    return model # 학습된 모델 반환

def voting_function(data, estimators, voting='hard', threshold=0.5):
    # 데이터셋 분할 # voting='hard'일 경우 threshold는 사용되지 않음
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # VotingClassifier 설정
    voting_clf = VotingClassifier(estimators=estimators, voting=voting)

    # 모델 학습
    voting_clf.fit(x_train, y_train)

    if voting == 'soft':
        # 소프트 보팅의 경우 확률 예측
        y_val_pred_proba = voting_clf.predict_proba(x_val)[:, 1]
        y_val_pred = (y_val_pred_proba >= threshold).astype(int)
    else:
        # 하드 보팅의 경우 직접 예측
        y_val_pred = voting_clf.predict(x_val)

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, zero_division=0)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'Voting Classifier로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

    return voting_clf  # 학습된 VotingClassifier 반환

def voting(preds_or_probs, method='soft', threshold=0.3):
    """
    하드 보팅 또는 소프트 보팅을 사용하여 최종 예측을 수행합니다.

    Parameters:
    preds_or_probs (list of np.array): 각 모델의 예측 배열 리스트 (하드 보팅) 또는 예측 확률 배열 리스트 (소프트 보팅)
    method (str): 'soft' 또는 'hard' 보팅 방법 선택
    threshold (float): 소프트 보팅 시 예측을 양성으로 간주할 확률 임계값

    Returns:
    np.array: 최종 예측 결과
    """
    if method == 'soft':
        # 소프트 보팅: 각 모델의 확률 평균 계산
        soft_voting_probs = np.mean(preds_or_probs, axis=0)
        # 최종 예측: 평균 확률에 대해 스레드 홀드 적용
        final_predictions = (soft_voting_probs >= threshold).astype(int)
    elif method == 'hard':
        # 하드 보팅: 각 모델의 예측을 모아서 다수결 원칙 적용
        preds = np.array(preds_or_probs)
        final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=preds)
    else:
        raise ValueError("method 인자는 'soft' 또는 'hard'여야 합니다.")
    
    return final_predictions

### 모델 학습

Dam 모델

In [11]:
base_estimator = DecisionTreeClassifier(
    max_depth=27,
    min_samples_split=4,
    min_samples_leaf=1,
    max_features=0.8675781747711193,
    # n_estimators=517,
    random_state=RANDOM_STATE
)

train_model_Dam = train_and_evaluate_model(
    'ada', train_data_dam
    , estimator=base_estimator
    , n_estimators=833
    , learning_rate=0.9817802538802249
    , random_state=RANDOM_STATE
)

ada 모델이 train_data_dam 데이터로 학습한 결과:
F1 Score: 0.228099173553719
---
Confusion Matrix:
[[7566   96]
 [ 371   69]]
---
Accuracy: 0.9423599111330536
Precision: 0.41818181818181815
Recall: 0.15681818181818183




AutoClave 모델

In [12]:
base_estimator = DecisionTreeClassifier(
    max_depth=15,
    min_samples_split=35,
    min_samples_leaf=2,
    max_features=0.16617258789907924,
    # n_estimators=517,
    random_state=RANDOM_STATE
)

train_model_AutoClave = train_and_evaluate_model(
    'ada', train_data_autoclave
    , estimator=base_estimator
    , n_estimators=118
    , learning_rate=0.4047552485696655
    , random_state=RANDOM_STATE
)

ada 모델이 train_data_autoclave 데이터로 학습한 결과:
F1 Score: 0.19047619047619044
---
Confusion Matrix:
[[6943  719]
 [ 318  122]]
---
Accuracy: 0.8720069118736115
Precision: 0.1450653983353151
Recall: 0.2772727272727273




Fill1 모델

In [13]:
base_estimator = DecisionTreeClassifier(
    max_depth=12,
    min_samples_split=27,
    min_samples_leaf=8,
    max_features=0.6030749544065899,
    # n_estimators=517,
    random_state=RANDOM_STATE
)

train_model_Fill1 = train_and_evaluate_model(
    'ada', train_data_fill1
    , estimator=base_estimator
    , n_estimators=400
    , learning_rate=0.48125695338518293
    , random_state=RANDOM_STATE
)

ada 모델이 train_data_fill1 데이터로 학습한 결과:
F1 Score: 0.19525065963060687
---
Confusion Matrix:
[[7418  244]
 [ 366   74]]
---
Accuracy: 0.9247099481609479
Precision: 0.23270440251572327
Recall: 0.16818181818181818




Fill2 모델

In [14]:
base_estimator = DecisionTreeClassifier(
    max_depth=13,
    min_samples_split=4,
    min_samples_leaf=2,
    max_features=0.8686759201566782,
    # n_estimators=517,
    random_state=RANDOM_STATE
)

train_model_Fill2 = train_and_evaluate_model(
    'ada', train_data_fill2
    , estimator=base_estimator
    , n_estimators=517
    , learning_rate=0.25710844249446485
    , random_state=RANDOM_STATE
)

ada 모델이 train_data_fill2 데이터로 학습한 결과:
F1 Score: 0.18980169971671387
---
Confusion Matrix:
[[7463  199]
 [ 373   67]]
---
Accuracy: 0.9294001481115773
Precision: 0.2518796992481203
Recall: 0.15227272727272728




전체공정 모델

In [15]:
# base_estimator = DecisionTreeClassifier(
#     max_depth=6,
#     min_samples_split=30,
#     min_samples_leaf=12,
#     max_features=0.8457354826252363,
#     # n_estimators=517,
#     random_state=RANDOM_STATE
# )

# train_model_All = train_and_evaluate_model(
#     'ada', train_data
#     , estimator=base_estimator
#     , n_estimators=541
#     , learning_rate=0.3430292944431681
#     , random_state=RANDOM_STATE
# )

ada 모델이 train_data 데이터로 학습한 결과:
F1 Score: 0.11070780399274045
---
Confusion Matrix:
[[ 815 6847]
 [  13  427]]
---
Accuracy: 0.15329548259688966
Precision: 0.0587022271102557
Recall: 0.9704545454545455




soft voting

In [49]:
# VotingClassifier 사용 
estimators = [
    ('dam', train_model_Dam)
    , ('autoclave', train_model_AutoClave)
    , ('fill1', train_model_Fill1)
    , ('fill2', train_model_Fill2)
#     , ('all', train_model_All)
]

# VotingClassifier 학습 및 평가
# voting_clf_hard = voting_function(train_data, estimators, voting='hard')
voting_clf_soft = voting_function(train_data, estimators, voting='soft', threshold=0.3)

Voting Classifier로 학습한 결과:
F1 Score: 0.19494584837545126
---
Confusion Matrix:
[[7602   60]
 [ 386   54]]
---
Accuracy: 0.9449518637373489
Precision: 0.47368421052631576
Recall: 0.12272727272727273




In [50]:
# VotingClassifier 학습 및 평가
voting_clf_soft = voting_function(train_data, estimators, voting='soft', threshold=0.24)

Voting Classifier로 학습한 결과:
F1 Score: 0.19298245614035087
---
Confusion Matrix:
[[7587   75]
 [ 385   55]]
---
Accuracy: 0.9432238953344854
Precision: 0.4230769230769231
Recall: 0.125




In [51]:
# VotingClassifier 학습 및 평가
voting_clf_soft = voting_function(train_data, estimators, voting='soft', threshold=0.26)

Voting Classifier로 학습한 결과:
F1 Score: 0.1904761904761905
---
Confusion Matrix:
[[7589   73]
 [ 386   54]]
---
Accuracy: 0.9433473216489756
Precision: 0.4251968503937008
Recall: 0.12272727272727273




In [52]:
# VotingClassifier 학습 및 평가
voting_clf_soft = voting_function(train_data, estimators, voting='soft', threshold=0.28)

Voting Classifier로 학습한 결과:
F1 Score: 0.19354838709677422
---
Confusion Matrix:
[[7598   64]
 [ 386   54]]
---
Accuracy: 0.9444581584793879
Precision: 0.4576271186440678
Recall: 0.12272727272727273




In [20]:
# VotingClassifier 학습 및 평가
voting_clf_soft = voting_function(train_data, estimators, voting='soft', threshold=0.32)

Voting Classifier로 학습한 결과:
F1 Score: 0.19285714285714287
---
Confusion Matrix:
[[7596   66]
 [ 386   54]]
---
Accuracy: 0.9442113058504074
Precision: 0.45
Recall: 0.12272727272727273




In [21]:
voting_clf_soft = voting_function(train_data, estimators, voting='soft', threshold=0.22)

Voting Classifier로 학습한 결과:
F1 Score: 0.20512820512820512
---
Confusion Matrix:
[[7542  120]
 [ 376   64]]
---
Accuracy: 0.9387805480128364
Precision: 0.34782608695652173
Recall: 0.14545454545454545




In [22]:
voting_clf_soft = voting_function(train_data, estimators, voting='soft', threshold=0.2)

KeyboardInterrupt: 

In [ ]:
voting_clf_soft = voting_function(train_data, estimators, voting='soft', threshold=0.29)

---

### 모델 학습(train 데이터 전체 학습)

위의 모델학습 코드에서 함수명만 바뀌고 들어가는 값들은 동일  
-> 위의 코드 복붙한다음 함수명만 바꿔주면 사용하기 편함  

In [54]:
base_estimator = DecisionTreeClassifier(
    max_depth=27,
    min_samples_split=4,
    min_samples_leaf=1,
    max_features=0.8675781747711193,
    random_state=RANDOM_STATE
)

train_model_Dam = fit_all_train_data_function(
    'ada', train_data_dam
    , estimator=base_estimator
    , n_estimators=833
    , learning_rate=0.9817802538802249
    , random_state=RANDOM_STATE
)

base_estimator = DecisionTreeClassifier(
    max_depth=15,
    min_samples_split=35,
    min_samples_leaf=2,
    max_features=0.16617258789907924,
    random_state=RANDOM_STATE
)

train_model_AutoClave = fit_all_train_data_function(
    'ada', train_data_autoclave
    , estimator=base_estimator
    , n_estimators=118
    , learning_rate=0.4047552485696655
    , random_state=RANDOM_STATE
)

base_estimator = DecisionTreeClassifier(
    max_depth=12,
    min_samples_split=27,
    min_samples_leaf=8,
    max_features=0.6030749544065899,
    random_state=RANDOM_STATE
)

train_model_Fill1 = fit_all_train_data_function(
    'ada', train_data_fill1
    , estimator=base_estimator
    , n_estimators=400
    , learning_rate=0.48125695338518293
    , random_state=RANDOM_STATE
)

base_estimator = DecisionTreeClassifier(
    max_depth=13,
    min_samples_split=4,
    min_samples_leaf=2,
    max_features=0.8686759201566782,
    random_state=RANDOM_STATE
)

train_model_Fill2 = fit_all_train_data_function(
    'ada', train_data_fill2
    , estimator=base_estimator
    , n_estimators=517
    , learning_rate=0.25710844249446485
    , random_state=RANDOM_STATE
)

# base_estimator = DecisionTreeClassifier(
#     max_depth=6,
#     min_samples_split=30,
#     min_samples_leaf=12,
#     max_features=0.8457354826252363,
#     random_state=RANDOM_STATE
# )

# train_model_All = fit_all_train_data_function(
#     'ada', train_data
#     , estimator=base_estimator
#     , n_estimators=541
#     , learning_rate=0.3430292944431681
#     , random_state=RANDOM_STATE
# )

ada 모델이 train_data_dam 데이터로 학습 완료
ada 모델이 train_data_autoclave 데이터로 학습 완료
ada 모델이 train_data_fill1 데이터로 학습 완료
ada 모델이 train_data_fill2 데이터로 학습 완료


---

In [55]:
# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)
x_test_all = test_data.drop(["target", "Set ID"], axis=1)

In [56]:
x_test_dam.fillna(0, inplace=True)
x_test_autoclave.fillna(0, inplace=True)
x_test_fill1.fillna(0, inplace=True)
x_test_fill2.fillna(0, inplace=True)
x_test_all.fillna(0, inplace=True)

Voting(hard)

4개의 모델중 다수의 모델이 판단한 값을 타겟값으로 가져감

In [57]:
'''
# 예측 리스트 (하드 보팅용)
preds = [
    model_Dam.predict(x_test_dam)
    , model_AutoClave.predict(x_test_autoclave)
    , model_Fill1.predict(x_test_fill1)
    , model_Fill2.predict(x_test_fill2)
    # , model_All.predict(x_test_all)
]

# 하드 보팅 결과
final_predictions = voting(preds, method='hard')
print(sum(final_predictions))
'''

"\n# 예측 리스트 (하드 보팅용)\npreds = [\n    model_Dam.predict(x_test_dam)\n    , model_AutoClave.predict(x_test_autoclave)\n    , model_Fill1.predict(x_test_fill1)\n    , model_Fill2.predict(x_test_fill2)\n    # , model_All.predict(x_test_all)\n]\n\n# 하드 보팅 결과\nfinal_predictions = voting(preds, method='hard')\nprint(sum(final_predictions))\n"

Voting(soft)

4개의 모델의 확률의 합에 대한 평균이 일정 수치(스레스홀드) 값을 넘으면 AbNormal 값을 가짐  

In [58]:
# 예측 확률 리스트 (소프트 보팅용)
probs = [
    train_model_Dam.predict_proba(x_test_dam)[:, 1]
    , train_model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , train_model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , train_model_Fill2.predict_proba(x_test_fill2)[:, 1]
#     , train_model_All.predict_proba(x_test_all)[:, 1]
]

# 소프트 보팅 결과
final_predictions = voting(probs, method='soft', threshold=0.3)
print(sum(final_predictions))

708


## 4. 제출하기


### 제출 파일 작성


In [38]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'submission.csv'

In [ ]:
df_sub['target'].value_counts()

In [ ]:
df_sub.head(10)

In [ ]:
## 제출 F1 score = 0.1990881

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**


모델을 통해 예측해 낸 정답지(csv) 파일의 결과값을 불러와 voting 하는 방식  
이때의 voting은 hard방식만 가능(예측확률은 저장되어있지 않음으로 soft 보팅은 불가능)  

In [ ]:
# import pandas as pd
# import numpy as np

# def read_submission_files(file_paths):
#     """
#     제출 파일을 읽어와서 예측 결과를 반환합니다.

#     Parameters:
#     file_paths (list of str): 제출 파일 경로 리스트

#     Returns:
#     list of np.array: 각 제출 파일의 예측 결과 리스트
#     """
#     predictions = []
#     for file_path in file_paths:
#         df = pd.read_csv(file_path)
#         preds = df['target'].apply(lambda x: 1 if x == 'AbNormal' else 0).values
#         predictions.append(preds)
#     return predictions

# def hard_voting(preds):
#     """
#     하드 보팅을 사용하여 최종 예측을 수행합니다.

#     Parameters:
#     preds (list of np.array): 각 모델의 예측 배열 리스트

#     Returns:
#     np.array: 최종 예측 결과
#     """
#     preds = np.array(preds)
#     final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=preds)
#     return final_predictions

submission_file 에서 csv 파일을 불러와 hard voting 하는 알고리즘  

In [ ]:
# import pandas as pd

# # 공통 경로
# common_path = "../submission_file/"

# # 제출 파일 이름 리스트
# file_names = [
#     "data0816_light_cutoff0.28_submission.csv",
#     "submission_score0.23.csv"
#     # 파일 추가 가능  <----- 파일 필요시 추가하세요!!
# ]

# # 경로를 추가하는 함수
# def add_common_path(file_names, common_path):
#     return [common_path + file_name for file_name in file_names]

# # 경로가 추가된 파일 리스트
# file_paths = add_common_path(file_names, common_path)

# # 제출 파일에서 예측 결과 읽어오기
# predictions = read_submission_files(file_paths)

# # 하드 보팅 결과
# final_predictions_hard = hard_voting(predictions)

# # 결과를 새로운 제출 파일로 저장할 파일 이름
# output_file_name = "final_submission.csv" # <----- 파일 이름을 변경하세요!!

# # 결과를 새로운 제출 파일로 저장
# df_sub = pd.read_csv(file_paths[0])
# df_sub["target"] = final_predictions_hard
# df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')
# df_sub.to_csv(output_file_name, index=False)

# print(f"최종 제출 파일이 '{output_file_name}'로 저장되었습니다.")

In [ ]:
# df_sub['target'].value_counts()

In [ ]:
# df_sub.head(10)